In [143]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 22050

In [144]:
import pyaudio
import wave
import sys

def record_to_file(filename, seconds=5):
    global FORMAT, CHANNELS, RATE
    RECORD_SECONDS = seconds
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    print("Start recording... ", end="")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)  
        frames.append(data)
    print("recorded", seconds, "second(s)")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [145]:
import argparse
import io
import google.cloud

from google.cloud import speech
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
findat = ""
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=22050,
        language_code='en-US')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        # print(u'Transcript: {}'.format(result.alternatives[0].transcript))
        data = (u'{}'.format(result.alternatives[0].transcript))
    return(data)

In [146]:
buffer = "buffer.wav"
record_to_file(buffer)
wf = wave.open(buffer, 'rb')
findat = transcribe_file(buffer)
transcribe_file(buffer)

Start recording... recorded 5 second(s)


'hello my name is Joe'

In [159]:
import argparse
import six

from google.cloud import translate
translate_client = translate.Client()
target = 'es'
# Text can also be a sequence of strings, in which case this method
# will return a sequence of results for each text.
result = translate_client.translate(
    findat, target_language=target)

trans_text = ((u'{}'.format(result['translatedText'])))

Hola, mi nombre es Joe


In [162]:
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()
synthesis_input = texttospeech.types.SynthesisInput(text=trans_text)

print("Genders:", texttospeech.enums.SsmlVoiceGender._member_names_)
voice = texttospeech.types.VoiceSelectionParams(
    language_code='en-US',
    ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)

print("Encodings:", texttospeech.enums.AudioEncoding._member_names_)
audio_config = texttospeech.types.AudioConfig(audio_encoding=texttospeech.enums.AudioEncoding.MP3)
response = client.synthesize_speech(synthesis_input, voice, audio_config)

with open('datasets/sound/chord.wav', 'wb') as out:
    # Write the response to the output file.
    out.write(response.audio_content)

Genders: ['SSML_VOICE_GENDER_UNSPECIFIED', 'MALE', 'FEMALE', 'NEUTRAL']
Encodings: ['AUDIO_ENCODING_UNSPECIFIED', 'LINEAR16', 'MP3', 'OGG_OPUS']
